# 前処理

In [2]:
import pandas as pd
import numpy as np

## データの読み込み

In [3]:
# ローカルファイルを読み込む
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")
# google colaboratory で実行する場合は以下を有効にする
# from google.colab import drive
# drive.mount('/content/drive')
# train_df = pd.read_csv("/content/drive/mydrive/signate_cup_2024_data/train.csv")
# test_df = pd.read_csv("/content/drive/mydrive/signate_cup_2024_data/test.csv")

In [8]:
train_df.columns

Index(['id', 'Age', 'TypeofContact', 'CityTier', 'DurationOfPitch',
       'Occupation', 'Gender', 'NumberOfPersonVisiting', 'NumberOfFollowups',
       'ProductPitched', 'PreferredPropertyStar', 'NumberOfTrips', 'Passport',
       'PitchSatisfactionScore', 'Designation', 'MonthlyIncome',
       'customer_info', 'ProdTaken'],
      dtype='object')

In [11]:
# 3カラムにNULL値が存在するかどうか確認
print("Designateion NULL count: {}".format(train_df["Designation"].isnull().sum()))
print("MonthlyIncome NULL count: {}".format(train_df["MonthlyIncome"].isnull().sum()))
print("customer_info NULL count: {}".format(train_df["customer_info"].isnull().sum()))

Designateion NULL count: 0
MonthlyIncome NULL count: 56
customer_info NULL count: 0


## Designationカラム

In [4]:
def convert_fullwidth_to_halfwidth_and_extract_invalid(
    df: pd.DataFrame,
    column_name: str,
) -> pd.DataFrame:
    """
    指定されたカラムに対して次の処理を行う:
    1. 全角英字を半角英字に変換
    2. それでもなおAからz以外の文字が含まれるユニークな値をリストとして返す
    
    Args:
        df (pd.DataFrame): 対象のデータフレーム
        column_name (str): 操作を行うカラムの名前
    
    Returns:
        pd.DataFrame: 修正後のデータフレーム
        list: 条件に合わないレコードのユニークな値
    """
    # 全角英字を半角英字に変換
    def convert_fullwidth_to_halfwidth(text: str) -> str:
        """
        全角英字を半角英字に変換するヘルパー関数

        Args:
            text (str): 入力文字列

        Returns:
            str: 半角に変換された文字列
        """
        return ''.join(
            chr(ord(char) - 65248) if 'Ａ' <= char <= 'Ｚ' or 'ａ' <= char <= 'ｚ' else char
            for char in text
        )

    # 対象カラムの全角英字を半角英字に変換
    df[column_name] = df[column_name].apply(lambda x: convert_fullwidth_to_halfwidth(x) if pd.notna(x) else x)

    # Aからz以外の文字を含む行のindexを取得
    invalid_indices = df[~df[column_name].str.match(r'^[A-Za-z]+$', na=False)].index

    # 条件に合わないレコードのユニークな値をリストとして取得
    unique_invalid_values = df.loc[invalid_indices, column_name].unique().tolist()

    return df, unique_invalid_values

In [13]:
# Designationカラムに対して処理を行う
train_df, unique_invalid_values = convert_fullwidth_to_halfwidth_and_extract_invalid(train_df, "Designation")
unique_invalid_values

['Senior Manager',
 'Senior Manage𝙧',
 'Μanager',
 'Sеnior Manager',
 'ΑVP',
 'АVP',
 'E×ecutive',
 'Mαnage𝙧',
 'Executiѵе',
 'Ѕenior Manager',
 'Managеr',
 'Еxecutivе',
 'Senior Μanαger',
 'Еxecutive',
 'Exеcutivе',
 'Exеcutive',
 'Senior Managе𝙧',
 'Manage𝙧',
 'Senio𝙧 Manager',
 'Manαger',
 'Μanage𝙧',
 'Manαgеr',
 'Senior Managеr',
 'Executivе',
 'Executiѵe',
 'Е×еcutive',
 'Տenior Μanager',
 'Exеcutiѵе',
 'Senior Manαger',
 'Mαnager',
 'Senior Mαnαger',
 'E×еcutiѵe',
 'Ѕenior Manαger',
 'Mαnαger',
 'Μanagеr',
 'Sеnior Managеr',
 'Տenior Manager',
 'Mαnagеr',
 'Еxеcutivе',
 'Senior Mαnager',
 'Е×ecutive']

In [6]:
# サンプルデータフレーム
data = pd.DataFrame(
        {
        "name": ["ＡＢＣ", "xyz", "123", "ＡＢＣＤＥ", "漢字", "ＡＢＣdef", "ＡＢＣ１"],
        "value": [1, 2, 3, 4, 5, 6, 7]
        }
)
convert_fullwidth_to_halfwidth_and_extract_invalid(data, "name")

(     name  value
 0     ABC      1
 1     xyz      2
 2     123      3
 3   ABCDE      4
 4      漢字      5
 5  ABCdef      6
 6    ABC１      7,
 ['123', '漢字', 'ABC１'])

## MonthlyIncomeカラム

## Customer_infoカラム